In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
import numpy as np
import pathlib
import matplotlib.pyplot as plt
from os.path import dirname, join as pjoin
import scipy.io as sio
import h5py
import hdf5storage
import random
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
from PIL import Image
from matplotlib.image import imread
import os
from tensorflow.keras import applications
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

Загрузим заранее подготовленный датасет:

In [2]:
DATASET_ROOT = '/home/maxim/Desktop/ml_dataset/sign-language-mnist/'

train_data = pd.read_csv(DATASET_ROOT + '/sign_mnist_train.csv')
test_data = pd.read_csv(DATASET_ROOT + '/sign_mnist_test.csv')
print('Train set: ' + str(train_data.shape))
print('Test set: ' + str(test_data.shape))

binarizer = LabelBinarizer()

train_labels = train_data['label'].values
train_labels = binarizer.fit_transform(train_labels)
train_data.drop('label', axis = 1, inplace = True)

test_labels = test_data['label'].values
test_labels = binarizer.fit_transform(test_labels)
test_data.drop('label', axis = 1, inplace = True)

print(str(train_labels.shape))
print(str(test_labels.shape))

out_count = test_labels.shape[1]

Train set: (27455, 785)
Test set: (7172, 785)
(27455, 24)
(7172, 24)


In [3]:
VALIDATION_SET_RATE = 0.05

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
IMAGE_CHANNELS = 1
BATCH_SIZE = 32
EPOCHS_COUNT = 5

train_values = train_data.values
test_values = test_data.values
print(train_values.shape)
print(test_values.shape)

(27455, 784)
(7172, 784)


Разобьём датасет на выборки:

In [4]:
# SPLIT ON SETS
x_train = np.asarray(train_values).reshape(train_values.shape[0], IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)
x_test = np.asarray(test_values).reshape(test_values.shape[0], IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)

x_train, x_validation, y_train, y_validation = train_test_split(
    x_train,
    train_labels,
    test_size = VALIDATION_SET_RATE, 
    stratify = train_labels
)
y_test = test_labels

print(x_train.shape)
print(x_validation.shape)
print(x_test.shape)

(26082, 28, 28, 1)
(1373, 28, 28, 1)
(7172, 28, 28, 1)


Преобразуем данные при помощи ImageDataGenerator:

In [5]:
# PUT IN IMAGE DATA GENERATOR

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
au_validation_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

au_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    horizontal_flip=True
)

data_loader = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
modified_image_shape = (32, 32, 3)

x_train_vgg = np.pad(
    x_train,
    ((0,0),(2,2),(2,2),(0,0)),
    'edge'
)
x_train_vgg = np.repeat(x_train_vgg, 3, axis=3)
train_flow_vgg = data_loader.flow(
    x_train_vgg,
    y_train,
    batch_size=BATCH_SIZE
)

x_test_vgg = np.pad(
    x_test,
    ((0,0),(2,2),(2,2),(0,0)),
    'edge'
)
x_test_vgg = np.repeat(x_test_vgg, 3, axis=3)
test_flow_vgg = data_loader.flow(
    x_test_vgg,
    y_test,
    batch_size=BATCH_SIZE
)
print(x_train_vgg.shape)
print(x_test_vgg.shape)

(26082, 32, 32, 3)
(7172, 32, 32, 3)


In [6]:
# DESCRIBE FLOWS

train_flow = image_generator.flow(x_train, y_train)
validation_flow = validation_image_generator.flow(x_validation, y_validation)
test_flow = test_image_generator.flow(x_test, y_test)
au_train_flow = au_image_generator.flow(x_train, y_train)
au_validation_flow = au_validation_image_generator.flow(x_validation, y_validation)
# vgg_flow = vgg_image_generator.flow(x_train, y_train)
# resnet_flow = resnet_image_generator.flow(x_train, y_train)

STEPS_PER_EPOCH_TRAIN = 600
AU_STEPS_PER_EPOCH_TRAIN = 600
VGG_STEPS_PER_EPOCH_TRAIN = 600
RESNET_STEPS_PER_EPOCH_TRAIN = 800
STEPS_PER_EPOCH_VALIDATION = 10
AU_STEPS_PER_EPOCH_VALIDATION = 10

Определим первую модель, данная модель состоит из 3х пар свёртка-пулинг и 3х полносвязных слоёв.

In [7]:
# DEFINE THE MODEL

input_layer = layers.Input(shape=((IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)))
conv_1 = layers.Conv2D(16, kernel_size=(3, 3), padding='valid')
conv_2 = layers.Conv2D(32, kernel_size=(3, 3), padding='valid')
conv_3 = layers.Conv2D(64, kernel_size=(3, 3), padding='valid')
mp_1 = layers.MaxPooling2D(strides=(2, 2))
mp_2 = layers.MaxPooling2D(strides=(2, 2))
mp_3 = layers.MaxPooling2D(strides=(2, 2))

fc_1 = layers.Dense(128, activation='relu')
fc_2 = layers.Dense(64, activation='relu')

out_layer = layers.Dense(out_count, activation='sigmoid')

model = keras.Sequential([
        input_layer,
        conv_1,
        mp_1,
        conv_2,
        mp_2,
        conv_3,
        mp_3,
        layers.Flatten(),
        fc_1,
        fc_2,
        out_layer
    ])
model.summary()

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0

Для упрощения отладки - модели будем сохранять:

In [8]:
keras.backend.clear_session()

if os.path.exists('three_conv.h5'):
    model = load_model('three_conv.h5')
else:
    history = model.fit_generator(
        train_flow,
        validation_data=validation_flow,
        steps_per_epoch=STEPS_PER_EPOCH_TRAIN,
        validation_steps=STEPS_PER_EPOCH_VALIDATION,
        epochs=EPOCHS_COUNT,
        verbose=1
    )
    model.save('three_conv.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 600 steps, validate for 10 steps
Epoch 1/5
600/600 [==============================] - 4s 6ms/step - loss: 1.6572 - accuracy: 0.4653 - val_loss: 0.5905 - val_accuracy: 0.8250
Epoch 2/5
600/600 [==============================] - 6s 9ms/step - loss: 0.3178 - accuracy: 0.8953 - val_loss: 0.1533 - val_accuracy: 0.9594
Epoch 3/5
600/600 [==============================] - 6s 10ms/step - loss: 0.0778 - accuracy: 0.9768 - val_loss: 0.0592 - val_accuracy: 0.9812
Epoch 4/5
600/600 [==============================] - 6s 10ms/step - loss: 0.0370 - accuracy: 0.9898 - val_loss: 0.0159 - val_accuracy: 0.9937
Epoch 5/5
600/600 [==============================] - 6s 10ms/step - loss: 0.0271 - accuracy: 0.9927 - val_loss: 0.0044 - val_accuracy: 1.0000


In [9]:
model.evaluate(test_flow)

  ...
    to  
  ['...']
225/225 [==============================] - 1s 3ms/step - loss: 0.5197 - accuracy: 0.8680


[0.519660881674952, 0.8679587]

Данная модель показывает неплохие данные данные - 86% точности на тестовой выборке.

Далее определим схожу с предыдущей модель, однако при её обучении будем использовать augumented data.

In [10]:
au_input_layer = layers.Input(shape=((IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)))
au_conv_1 = layers.Conv2D(16, kernel_size=(3, 3), padding='valid')
au_conv_2 = layers.Conv2D(32, kernel_size=(3, 3), padding='valid')
au_conv_3 = layers.Conv2D(64, kernel_size=(3, 3), padding='valid')
au_mp_1 = layers.MaxPooling2D(strides=(2, 2))
au_mp_2 = layers.MaxPooling2D(strides=(2, 2))
au_mp_3 = layers.MaxPooling2D(strides=(2, 2))

au_fc_1 = layers.Dense(128, activation='relu')
au_fc_2 = layers.Dense(64, activation='relu')

au_out_layer = layers.Dense(out_count, activation='sigmoid')

au_model = keras.Sequential([
        au_input_layer,
        au_conv_1,
        au_mp_1,
        au_conv_2,
        au_mp_2,
        au_conv_3,
        au_mp_3,
        layers.Flatten(),
        au_fc_1,
        au_fc_2,
        au_out_layer
    ])
au_model.summary()

au_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0

Для упрощения отладки, будем сохранять модели.

In [11]:
if os.path.exists('three_conv_au.h5'):
    au_model = load_model('three_conv_au.h5')
else:
    au_history = au_model.fit_generator(
        au_train_flow,
        validation_data=au_validation_flow,
        steps_per_epoch=AU_STEPS_PER_EPOCH_TRAIN,
        validation_steps=AU_STEPS_PER_EPOCH_VALIDATION,
        epochs=EPOCHS_COUNT,
        verbose=1
    )
    au_model.save('three_conv_au.h5')

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 600 steps, validate for 10 steps
Epoch 1/5
600/600 [==============================] - 9s 15ms/step - loss: 2.8143 - accuracy: 0.1341 - val_loss: 1.8625 - val_accuracy: 0.3656
Epoch 2/5
600/600 [==============================] - 8s 14ms/step - loss: 1.7719 - accuracy: 0.4107 - val_loss: 1.1812 - val_accuracy: 0.6094
Epoch 3/5
600/600 [==============================] - 9s 14ms/step - loss: 1.2785 - accuracy: 0.5640 - val_loss: 0.8722 - val_accuracy: 0.7125
Epoch 4/5
600/600 [==============================] - 8s 14ms/step - loss: 1.0612 - accuracy: 0.6378 - val_loss: 0.7069 - val_accuracy: 0.7594
Epoch 5/5
600/600 [==============================] - 8s 14ms/step - loss: 0.9116 - accuracy: 0.6908 - val_loss: 0.6182 - val_accuracy: 0.7875


In [12]:
# On augumented 3-conv
au_model.evaluate(test_flow)

  ...
    to  
  ['...']
225/225 [==============================] - 1s 3ms/step - loss: 0.8112 - accuracy: 0.6924


[0.811235894229677, 0.69241494]

Как видно, данная модель показывает себя чуть хуже в процентном соотношении, однако стоит отметить, что она обучается медленнее и, вероятно, при использовании большего количества эпох результат модели будет лучше. Однако для чистоты сравнения - будем использовать одинаковое количество эпох.

Следующей моделью будет VGG-16.

In [13]:
# VGG-16
keras.backend.clear_session()
    
# Load model and mark pretrained layers as not trainable
vgg16 = applications.VGG16(include_top=False, input_shape=(32, 32, 3))
for layer in vgg16.layers:
    layer.trainable = False
        
# Forward pretrained model output into new layers
x = layers.Flatten()(vgg16.layers[-1].output)
x = layers.Dense(128, activation='relu')(x)
output = layers.Dense(out_count, activation='sigmoid')(x)
    
# Compile model
vgg16 = keras.Model(inputs=vgg16.inputs, outputs=output)
vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [14]:
if os.path.exists('vgg16.h5'):
    vgg16 = load_model('vgg16.h5')
else:
    vgg_history = vgg16.fit_generator(
        train_flow_vgg,
        steps_per_epoch=VGG_STEPS_PER_EPOCH_TRAIN,
        epochs=3,
        verbose=1
    )
    vgg16.save('vgg16.h5')

  ...
    to  
  ['...']
Train for 600 steps
Epoch 1/3
600/600 [==============================] - 106s 176ms/step - loss: 1.1921 - accuracy: 0.6651
Epoch 2/3
600/600 [==============================] - 109s 182ms/step - loss: 0.4174 - accuracy: 0.8937
Epoch 3/3
600/600 [==============================] - 106s 176ms/step - loss: 0.2405 - accuracy: 0.9479


Как видно из процесса обучения - данная модель обучается медленнее остальных, однако стоит отметить, что она выдаёт хорошую точность на обучающей выборке.

In [15]:
vgg16.evaluate(test_flow_vgg)

  ...
    to  
  ['...']
225/225 [==============================] - 40s 178ms/step - loss: 0.4044 - accuracy: 0.8610


[0.40436958938837053, 0.8609872]

На тестовых данных данная модель показала себя так же неплохо.

Можно отметить, что все модели показали себя достаточно хорошо, выдавая точность выше 80%. Где худший результат был у модели использующей аугументированные данных, возможно, из за специфики распознавания, где не особо допускаются координальные изменения изображений.